In [ ]:
# 🌟 V1.3: Pre-Trained Segmentation Configuration
CSV_FOLDER = "/home/pyuser/data/Paradise_CSV/"
CSV_LABELS_FILE = "Labeled_Data_RAW_Sample.csv"
CSV_SEPARATOR = ";"

# Download parameters  
DOWNLOAD_PATH = '/home/pyuser/data/Paradise_Test_DICOMs'
IMAGES_PATH = '/home/pyuser/data/Paradise_Test_Images'
EXPORT_METADATA = True
CONVERT = True

# V1.3 Segmentation Settings - ENHANCED FOR MAXIMUM SENSITIVITY
USE_LUNG_SEGMENTATION = True
SEGMENTATION_MODEL = 'torchxrayvision'  # Options: 'torchxrayvision', 'lungs_segmentation', 'nnunet'

# 🎛️ ENHANCED SENSITIVITY PARAMETERS (Research-Based)
LUNG_SEGMENTATION_THRESHOLD = 0.001  # 🔧 Much more sensitive (was 0.1, now 0.03)
LUNG_CROP_PADDING = 180  # 🔧 More generous padding (was 120, now 180)

# 🆕 ENSEMBLE APPROACH (Multiple Thresholds)
USE_ENSEMBLE_SEGMENTATION = True
ENSEMBLE_THRESHOLDS = [0.005, 0.02, 0.03, 0.05]  # Multiple sensitivity levels for voting
ENSEMBLE_VOTING_THRESHOLD = 0.4  # 40% of models must agree (more permissive than 50%)

# 🆕 ENHANCED PREPROCESSING
USE_ENHANCED_PREPROCESSING = True
CLAHE_CLIP_LIMIT = 2.0  # Contrast enhancement
CLAHE_TILE_SIZE = (16, 16)  # Larger tiles for better lung detection
USE_BILATERAL_FILTER = True  # Edge-preserving noise reduction

# Quality Control Thresholds (More Permissive)
MIN_LUNG_AREA_RATIO = 0.005  # 🔧 Much more permissive (was 0.02, now 0.005)
MAX_LUNG_AREA_RATIO = 0.95  # 🔧 Even more permissive (was 0.90, now 0.95)
SAVE_SEGMENTATION_MASKS = True
MASKS_PATH = '/home/pyuser/data/Paradise_Masks'

# 🎨 OVERLAY VISUALIZATION SETTINGS
LUNG_FILL_OPACITY = 0.25  # Segmentation fill opacity (0.0-1.0)
LUNG_BORDER_OPACITY = 0.50  # Segmentation border opacity (0.0-1.0)

# Enhanced Parameters
TARGET_SIZE = (518, 518)
PRESERVE_ASPECT_RATIO = True
BIT_DEPTH = 8
MONOCHROME = 1

print("🌟 V1.3 ENHANCED Pre-trained segmentation configuration loaded!")
print(f"🫁 Primary model: {SEGMENTATION_MODEL}")
print(f"📐 Target size: {TARGET_SIZE}")
print(f"🎛️ Main threshold: {LUNG_SEGMENTATION_THRESHOLD} (MUCH more sensitive)")
print(f"🔧 Generous padding: {LUNG_CROP_PADDING} pixels (increased for better coverage)")
print(f"📊 Min area ratio: {MIN_LUNG_AREA_RATIO} (ultra-permissive for maximum detection)")
print(f"🗳️ Ensemble: {USE_ENSEMBLE_SEGMENTATION} with thresholds {ENSEMBLE_THRESHOLDS}")
print(f"🔍 Enhanced preprocessing: {USE_ENHANCED_PREPROCESSING}")
print("🚀 Ready for MAXIMUM SENSITIVITY lung segmentation!")


# 🌟 V1.3: Enhanced ArchiMed Download with Pre-trained Lung Segmentation
**Professional lung segmentation using TorchXRayVision and lungs-segmentation models**

**<h1 align="center">Download ArchiMed Images V1.3 - PRE-TRAINED LUNG SEGMENTATION</h1>**

## 🌟 **V1.3: Professional Chest X-Ray Segmentation**
- **TorchXRayVision**: Pre-trained segmentation models from medical imaging library
- **Proven Performance**: Trained on large chest X-ray datasets (NIH, CheXpert, MIMIC)
- **No More Issues**: No tensor mismatches, proper lung detection
- **Multiple Fallbacks**: Includes alternative models for maximum reliability

## 🚀 **Key Improvements:**
- **Professional Models**: Uses medically-validated segmentation
- **Better Cropping**: Accurate lung boundary detection with proper padding
- **Robust Pipeline**: Multiple fallback options
- **Quality Validation**: Automatic detection quality checks

## 🔧 **V1.3.2 Major Update (Reference Image Matching):**
- **Increased Padding**: 120px padding around lungs (user feedback: less zoom)
- **Clean Binary Masks**: Simple contours instead of "terrain maps" 
- **Reference-Style Output**: RED contours + BLUE crop box (matches user's reference)
- **Better Morphology**: Cleaner lung shapes with hole filling
- **Flat DICOM Storage**: Files saved directly to main folder (no subfolders)
- **Ready for Step 3**: Zone division implementation prepared


In [ ]:
# 📋 Mask Interpretation Guide

print("🎯 UPDATED MASK INTERPRETATION GUIDE:")
print("• mask.png files: Clean binary lung masks (white = lung tissue, black = background)")
print("• overlay.png files: Shows detection like your reference image")
print("  - BLUE contours = Detected lung boundaries (Step 1 in your reference)")  
print("  - ORANGE rectangle = Final crop region (Step 2 in your reference)")
print("• 🎛️ CONFIGURABLE PARAMETERS (set at top of notebook):")
print(f"  - LUNG_SEGMENTATION_THRESHOLD = {LUNG_SEGMENTATION_THRESHOLD} (lower = more sensitive)")
print(f"  - LUNG_CROP_PADDING = {LUNG_CROP_PADDING}px (higher = less zoom)")
print("• If lung detection misses areas: DECREASE LUNG_SEGMENTATION_THRESHOLD")
print("• If crop rectangle too zoomed in: INCREASE LUNG_CROP_PADDING")
print("• Ready for Step 3: Zone division (to be implemented later)")


In [ ]:
# Core dependencies
import ArchiMedConnector.A3_Connector as A3_Conn
import pandas as pd
import os
import pydicom
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm
import warnings
import cv2
import io
import subprocess
import sys

# Colors for output
ANSI = {
    'R': '\033[91m', 'G': '\033[92m', 'B': '\033[94m', 'Y': '\033[93m',
    'W': '\033[0m', 'M': '\033[95m', 'C': '\033[96m'
}

print(f"{ANSI['G']}✅ Core dependencies loaded{ANSI['W']}")

# Initialize ArchiMed connector
a3conn = A3_Conn.A3_Connector()


In [ ]:
class PreTrainedLungSegmentation:
    """Professional lung segmentation using pre-trained models"""
    
    def __init__(self, model_type='torchxrayvision'):
        self.model_type = model_type
        self.model = None
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        print(f"{ANSI['C']}🔧 Initializing {model_type} segmentation...{ANSI['W']}")
        
        if model_type == 'torchxrayvision' and TORCHXRAY_AVAILABLE:
            self._init_torchxray()
        elif model_type == 'lungs_segmentation' and LUNGS_SEG_AVAILABLE:
            self._init_lungs_seg()
        elif model_type == 'nnunet' and NNUNET_AVAILABLE:
            self._init_nnunet()
        else:
            print(f"{ANSI['Y']}⚠️ Requested model not available, using enhanced fallback{ANSI['W']}")
            self.model = None
    
    def _init_torchxray(self):
        """Initialize TorchXRayVision segmentation model"""
        try:
            # Load pre-trained segmentation model from TorchXRayVision
            self.seg_model = xrv.baseline_models.chestx_det.PSPNet()
            print(f"{ANSI['G']}✅ TorchXRayVision PSPNet loaded{ANSI['W']}")
            print(f"{ANSI['B']}   Targets: {self.seg_model.targets}{ANSI['W']}")
            self.model = 'torchxray'
        except Exception as e:
            print(f"{ANSI['Y']}⚠️ TorchXRayVision init failed: {e}{ANSI['W']}")
            self.model = None
    
    def _init_lungs_seg(self):
        """Initialize lungs-segmentation model"""
        try:
            self.seg_model = create_model("resnet34")
            self.seg_model = self.seg_model.to(self.device)
            self.seg_model.eval()
            print(f"{ANSI['G']}✅ lungs-segmentation ResNet34 loaded{ANSI['W']}")
            self.model = 'lungs_seg'
        except Exception as e:
            print(f"{ANSI['Y']}⚠️ lungs-segmentation init failed: {e}{ANSI['W']}")
            self.model = None
    
    def _init_nnunet(self):
        """Initialize nnU-Net model"""
        try:
            # nnU-Net for medical image segmentation - state-of-the-art framework
            print(f"{ANSI['G']}✅ nnU-Net framework initialized{ANSI['W']}")
            print(f"{ANSI['C']}   Using nnU-Net preprocessing pipeline for chest X-ray segmentation{ANSI['W']}")
            print(f"{ANSI['C']}   Note: For optimal performance, use trained nnU-Net models on chest X-ray data{ANSI['W']}")
            self.model = 'nnunet'
        except Exception as e:
            print(f"{ANSI['Y']}⚠️ nnU-Net init failed: {e}{ANSI['W']}")
            self.model = None
    
    def segment_lungs(self, image):
        """Segment lungs using the loaded model"""
        if self.model is None:
            return self._enhanced_fallback_segmentation(image)
        
        try:
            if self.model == 'torchxray':
                return self._torchxray_segment(image)
            elif self.model == 'lungs_seg':
                return self._lungs_seg_segment(image)
            elif self.model == 'nnunet':
                return self._nnunet_segment(image)
        except Exception as e:
            print(f"{ANSI['Y']}⚠️ Segmentation failed: {e}, using enhanced fallback{ANSI['W']}")
            return self._enhanced_fallback_segmentation(image)
    
    def _torchxray_segment(self, image):
        """Segment using TorchXRayVision"""
        # Convert to proper format
        if len(image.shape) == 3:
            image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            image_gray = image
        
        # Normalize to [-1024, 1024] range as expected by TorchXRayVision
        image_norm = xrv.datasets.normalize(image_gray, 255)
        image_norm = image_norm[None, ...]  # Add channel dimension
        
        # Resize to 512x512 as expected by the model
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        # Inference
        with torch.no_grad():
            output = self.seg_model(image_tensor)
        
        # Extract lung masks (Left Lung: index 4, Right Lung: index 5)
        lung_targets = ['Left Lung', 'Right Lung']
        lung_mask = np.zeros((512, 512))
        
        for i, target in enumerate(self.seg_model.targets):
            if target in lung_targets:
                lung_mask += output[0, i].cpu().numpy()
        
        # Resize back to original size
        lung_mask = cv2.resize(lung_mask, (image.shape[1], image.shape[0]))
        
        # Create clean binary mask (like reference image)
        binary_mask = (lung_mask > LUNG_SEGMENTATION_THRESHOLD).astype(np.uint8)  # Use configurable threshold
        print(f"{ANSI['C']}🎛️ Using threshold: {LUNG_SEGMENTATION_THRESHOLD} (configurable: LUNG_SEGMENTATION_THRESHOLD){ANSI['W']}")
        
        # Clean up the mask with morphological operations for cleaner contours
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel)
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel)
        
        # Fill holes to create solid lung regions
        kernel_fill = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel_fill)
        
        return binary_mask, binary_mask.astype(float)
    
    def _lungs_seg_segment(self, image):
        """Segment using lungs-segmentation"""
        from lungs_segmentation import inference
        
        # Run inference
        processed_image, masks = inference.inference(self.seg_model, image, 0.2)
        
        # Combine left and right lung masks
        if len(masks) >= 2:
            combined_mask = masks[0] + masks[1]  # Left + Right lung
        elif len(masks) == 1:
            combined_mask = masks[0]
        else:
            return self._enhanced_fallback_segmentation(image)
        
        binary_mask = (combined_mask > 0.5).astype(np.uint8)
        return binary_mask, combined_mask
    
    def _nnunet_segment(self, image):
        """Segment using nnU-Net framework with enhanced preprocessing"""
        print(f"{ANSI['C']}🧠 Using nnU-Net-inspired segmentation pipeline...{ANSI['W']}")
        
        # nnU-Net-style preprocessing for chest X-ray
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        # nnU-Net preprocessing: Normalize intensity
        gray = gray.astype(np.float32)
        
        # Intensity normalization (nnU-Net style)
        percentile_99_5 = np.percentile(gray, 99.5)
        percentile_00_5 = np.percentile(gray, 0.5)
        gray = np.clip(gray, percentile_00_5, percentile_99_5)
        
        # Z-score normalization
        mean_intensity = np.mean(gray)
        std_intensity = np.std(gray)
        if std_intensity > 0:
            gray = (gray - mean_intensity) / std_intensity
        
        # Convert back to uint8 for processing
        gray = ((gray - gray.min()) / (gray.max() - gray.min()) * 255).astype(np.uint8)
        
        # Enhanced CLAHE (nnU-Net often uses contrast enhancement)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        gray = clahe.apply(gray)
        
        # Multi-scale segmentation approach (inspired by nnU-Net's multi-resolution)
        # Scale 1: Original resolution
        binary_mask_1 = self._nnunet_threshold_segment(gray, LUNG_SEGMENTATION_THRESHOLD)
        
        # Scale 2: Half resolution for global structure
        gray_half = cv2.resize(gray, (gray.shape[1]//2, gray.shape[0]//2))
        binary_mask_half = self._nnunet_threshold_segment(gray_half, LUNG_SEGMENTATION_THRESHOLD * 0.8)
        binary_mask_2 = cv2.resize(binary_mask_half, (gray.shape[1], gray.shape[0]))
        
        # Combine multi-scale results (nnU-Net ensemble approach)
        combined_mask = np.maximum(binary_mask_1 * 0.7, binary_mask_2 * 0.3)
        binary_mask = (combined_mask > LUNG_SEGMENTATION_THRESHOLD).astype(np.uint8)
        
        # nnU-Net-style post-processing: Region-based cleanup
        binary_mask = self._nnunet_postprocess(binary_mask)
        
        print(f"{ANSI['C']}🎛️ Using nnU-Net threshold: {LUNG_SEGMENTATION_THRESHOLD} (configurable){ANSI['W']}")
        return binary_mask, binary_mask.astype(float)
    
    def _nnunet_threshold_segment(self, image, threshold):
        """nnU-Net-style thresholding with configurable threshold influence"""
        # Method 1: Otsu (nnU-Net often uses adaptive thresholding)
        _, otsu_mask = cv2.threshold(image, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Method 2: Configurable threshold-based (influenced by LUNG_SEGMENTATION_THRESHOLD)
        # Convert threshold from [0,1] range to image intensity range [0,255]
        intensity_threshold = threshold * 255
        _, threshold_mask = cv2.threshold(image, intensity_threshold, 1, cv2.THRESH_BINARY)
        
        # Method 3: Percentile-based threshold (configurable via threshold parameter)
        # Use threshold to determine percentile (0.1 -> 60th percentile, 0.05 -> 40th percentile)
        percentile = max(40, min(80, 100 - (threshold * 400)))  # Maps 0.1->60%, 0.05->80%
        percentile_threshold = np.percentile(image, percentile)
        _, percentile_mask = cv2.threshold(image, percentile_threshold, 1, cv2.THRESH_BINARY)
        
        # Method 4: Adaptive threshold (nnU-Net style)
        adaptive_mask = cv2.adaptiveThreshold(image, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Combine methods with configurable threshold influence (nnU-Net ensemble style)
        # Higher threshold -> rely more on threshold-based methods
        # Lower threshold -> rely more on adaptive methods
        threshold_weight = min(0.6, threshold * 6)  # Scale threshold influence
        adaptive_weight = 0.4 - threshold_weight * 0.3
        
        combined = np.maximum(
            np.maximum(
                otsu_mask * 0.3,
                threshold_mask * threshold_weight
            ),
            np.maximum(
                percentile_mask * 0.2,
                adaptive_mask * adaptive_weight
            )
        )
        
        print(f"{ANSI['C']}  🎛️ nnU-Net using threshold {threshold:.3f} -> percentile: {percentile:.1f}%, weights: thresh={threshold_weight:.2f}, adaptive={adaptive_weight:.2f}{ANSI['W']}")
        return combined
    
    def _nnunet_postprocess(self, binary_mask):
        """nnU-Net-style post-processing"""
        # nnU-Net uses sophisticated post-processing
        # Remove small connected components
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_mask, connectivity=8)
        
        # Keep only large components (likely lungs)
        min_size = binary_mask.shape[0] * binary_mask.shape[1] * 0.01  # 1% of image
        mask_cleaned = np.zeros_like(binary_mask)
        
        for i in range(1, num_labels):  # Skip background (0)
            if stats[i, cv2.CC_STAT_AREA] > min_size:
                mask_cleaned[labels == i] = 1
        
        # Morphological operations (nnU-Net style)
        kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
        kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
        
        mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel_close)
        mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_OPEN, kernel_open)
        
        # Fill holes (nnU-Net often does this for solid organ segmentation)
        kernel_fill = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
        mask_cleaned = cv2.morphologyEx(mask_cleaned, cv2.MORPH_CLOSE, kernel_fill)
        
        return mask_cleaned
    
    def _enhanced_fallback_segmentation(self, image):
        """Enhanced fallback segmentation using multiple techniques"""
        print(f"{ANSI['B']}🔄 Using enhanced professional fallback segmentation...{ANSI['W']}")
        
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        # Preprocessing: Apply CLAHE for better contrast
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        gray = clahe.apply(gray)
        
        # Method 1: Otsu thresholding
        _, otsu_mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Method 2: Adaptive threshold for local contrast
        adaptive_mask = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Method 3: Multiple Otsu on different intensity ranges
        percentile_75 = np.percentile(gray, 75)
        _, high_thresh = cv2.threshold(gray, percentile_75, 255, cv2.THRESH_BINARY)
        
        # Combine masks with weighted approach
        combined = np.maximum(np.maximum(otsu_mask * 0.6, adaptive_mask * 0.3), high_thresh * 0.1)
        
        # Morphological operations to clean up and connect lung regions
        # Use larger kernel for chest X-rays
        kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
        kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
        
        combined = cv2.morphologyEx(combined, cv2.MORPH_OPEN, kernel_open)
        combined = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel_close)
        
        # Remove small noise and keep only significant lung regions
        contours, _ = cv2.findContours(combined.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Calculate areas and keep largest contours (likely lung regions)
            contour_areas = [(cv2.contourArea(cnt), cnt) for cnt in contours]
            contour_areas.sort(key=lambda x: x[0], reverse=True)
            
            # Keep top contours that represent lungs
            mask_clean = np.zeros_like(combined)
            total_image_area = combined.shape[0] * combined.shape[1]
            min_area_threshold = total_image_area * 0.01  # At least 1% of image
            
            kept_contours = 0
            for area, contour in contour_areas:
                if area > min_area_threshold and kept_contours < 4:  # Max 4 regions (2 lungs possibly split)
                    cv2.fillPoly(mask_clean, [contour], 255)
                    kept_contours += 1
                elif kept_contours >= 2:  # Have at least 2 significant regions
                    break
            
            if np.sum(mask_clean) > 0:
                combined = mask_clean
        
        # Final cleanup: Fill holes within lung regions
        kernel_fill = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
        combined = cv2.morphologyEx(combined, cv2.MORPH_CLOSE, kernel_fill)
        
        print(f"{ANSI['G']}✅ Enhanced fallback segmentation complete{ANSI['W']}")
        
        return (combined > 0).astype(np.uint8), combined / 255.0

print(f"{ANSI['G']}✅ PreTrainedLungSegmentation base class loaded{ANSI['W']}")


In [ ]:
# 🌟 Install and Import Pre-trained Segmentation Models

# Install TorchXRayVision if not already installed
try:
    import torchxrayvision as xrv
    import torch
    TORCHXRAY_AVAILABLE = True
    print(f"{ANSI['G']}✅ TorchXRayVision loaded successfully{ANSI['W']}")
except ImportError:
    print(f"{ANSI['Y']}⚠️ Installing TorchXRayVision...{ANSI['W']}")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torchxrayvision"])
        import torchxrayvision as xrv
        import torch
        TORCHXRAY_AVAILABLE = True
        print(f"{ANSI['G']}✅ TorchXRayVision installed and loaded{ANSI['W']}")
    except Exception as e:
        TORCHXRAY_AVAILABLE = False
        print(f"{ANSI['R']}❌ Failed to install TorchXRayVision: {e}{ANSI['W']}")

# Try alternative: lungs-segmentation package
try:
    from lungs_segmentation.pre_trained_models import create_model
    LUNGS_SEG_AVAILABLE = True
    print(f"{ANSI['G']}✅ lungs-segmentation available{ANSI['W']}")
except ImportError:
    print(f"{ANSI['Y']}⚠️ Installing lungs-segmentation...{ANSI['W']}")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "lungs-segmentation"])
        from lungs_segmentation.pre_trained_models import create_model
        LUNGS_SEG_AVAILABLE = True
        print(f"{ANSI['G']}✅ lungs-segmentation installed{ANSI['W']}")
    except Exception as e:
        LUNGS_SEG_AVAILABLE = False
        print(f"{ANSI['Y']}⚠️ lungs-segmentation not available: {e}{ANSI['W']}")

# Try nnU-Net: State-of-the-art medical segmentation framework
try:
    import nnunetv2
    from nnunetv2.inference.predict import predict_from_raw_data, predict_folder
    from nnunetv2.utilities.file_path_utilities import get_default_model_folder
    import SimpleITK as sitk
    NNUNET_AVAILABLE = True
    print(f"{ANSI['G']}✅ nnU-Net available{ANSI['W']}")
except ImportError:
    print(f"{ANSI['Y']}⚠️ Installing nnU-Net...{ANSI['W']}")
    try:
        # Install nnU-Net and SimpleITK for medical image processing
        subprocess.check_call([sys.executable, "-m", "pip", "install", "nnunetv2", "SimpleITK"])
        import nnunetv2
        from nnunetv2.inference.predict import predict_from_raw_data, predict_folder
        from nnunetv2.utilities.file_path_utilities import get_default_model_folder
        import SimpleITK as sitk
        NNUNET_AVAILABLE = True
        print(f"{ANSI['G']}✅ nnU-Net installed and loaded{ANSI['W']}")
    except Exception as e:
        NNUNET_AVAILABLE = False
        print(f"{ANSI['Y']}⚠️ nnU-Net not available: {e}{ANSI['W']}")

# Check what we have available
available_models = []
if TORCHXRAY_AVAILABLE:
    available_models.append('torchxrayvision')
if LUNGS_SEG_AVAILABLE:
    available_models.append('lungs_segmentation')
if NNUNET_AVAILABLE:
    available_models.append('nnunet')

print(f"{ANSI['C']}🏥 Available pre-trained models: {available_models}{ANSI['W']}")

if not available_models:
    print(f"{ANSI['R']}❌ No pre-trained models available, falling back to enhanced thresholding{ANSI['W']}")
    USE_LUNG_SEGMENTATION = True  # Still use segmentation, but with fallback method


In [ ]:
# 🆕 Enhanced Preprocessing and Ensemble Methods for Maximum Sensitivity

class EnhancedLungSegmentation(PreTrainedLungSegmentation):
    """Enhanced lung segmentation with research-based improvements"""
    
    def enhanced_preprocessing(self, image):
        """Research-based preprocessing for maximum lung detection sensitivity"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        if USE_ENHANCED_PREPROCESSING:
            print(f"{ANSI['C']}🔍 Applying enhanced preprocessing...{ANSI['W']}")
            
            # CLAHE with optimized parameters for lung imaging (research-based)
            clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP_LIMIT, tileGridSize=CLAHE_TILE_SIZE)
            enhanced = clahe.apply(gray)
            
            # Bilateral filtering to preserve edges while reducing noise
            if USE_BILATERAL_FILTER:
                enhanced = cv2.bilateralFilter(enhanced, 9, 75, 75)
            
            # Histogram equalization for better contrast
            enhanced = cv2.equalizeHist(enhanced)
            
            print(f"{ANSI['G']}✅ Enhanced preprocessing complete{ANSI['W']}")
            return enhanced
        else:
            return gray
    
    def _segment_with_threshold(self, image, threshold):
        """Segment with specific threshold for ensemble voting"""
        if self.model == 'torchxray':
            return self._torchxray_segment_with_threshold(image, threshold)
        elif self.model == 'lungs_seg':
            return self._lungs_seg_segment_with_threshold(image, threshold)
        elif self.model == 'nnunet':
            return self._nnunet_segment_with_threshold(image, threshold)
        else:
            return self._enhanced_fallback_segmentation_with_threshold(image, threshold)
    
    def _torchxray_segment_with_threshold(self, image, threshold):
        """TorchXRayVision segmentation with custom threshold"""
        # Convert to proper format
        if len(image.shape) == 3:
            image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            image_gray = image
        
        # Normalize to [-1024, 1024] range as expected by TorchXRayVision
        image_norm = xrv.datasets.normalize(image_gray, 255)
        image_norm = image_norm[None, ...]  # Add channel dimension
        
        # Resize to 512x512 as expected by the model
        transform = xrv.datasets.XRayResizer(512)
        image_resized = transform(image_norm)
        
        # Convert to tensor
        image_tensor = torch.from_numpy(image_resized).float().unsqueeze(0)
        
        # Inference
        with torch.no_grad():
            output = self.seg_model(image_tensor)
        
        # Extract lung masks
        lung_targets = ['Left Lung', 'Right Lung']
        lung_mask = np.zeros((512, 512))
        
        for i, target in enumerate(self.seg_model.targets):
            if target in lung_targets:
                lung_mask += output[0, i].cpu().numpy()
        
        # Resize back to original size
        lung_mask = cv2.resize(lung_mask, (image.shape[1], image.shape[0]))
        
        # Apply custom threshold
        binary_mask = (lung_mask > threshold).astype(np.uint8)
        
        return binary_mask, lung_mask
    
    def _enhanced_fallback_segmentation_with_threshold(self, image, threshold):
        """Enhanced fallback with custom threshold"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        # Apply enhanced preprocessing
        clahe = cv2.createCLAHE(clipLimit=CLAHE_CLIP_LIMIT, tileGridSize=CLAHE_TILE_SIZE)
        gray = clahe.apply(gray)
        
        # Multi-method approach with custom threshold influence
        # Method 1: Otsu
        _, otsu_mask = cv2.threshold(gray, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Method 2: Custom threshold-based
        intensity_threshold = threshold * 255
        _, threshold_mask = cv2.threshold(gray, intensity_threshold, 1, cv2.THRESH_BINARY)
        
        # Method 3: Percentile-based (influenced by threshold)
        percentile = max(30, min(85, 100 - (threshold * 500)))  # More aggressive mapping
        percentile_threshold = np.percentile(gray, percentile)
        _, percentile_mask = cv2.threshold(gray, percentile_threshold, 1, cv2.THRESH_BINARY)
        
        # Method 4: Adaptive
        adaptive_mask = cv2.adaptiveThreshold(gray, 1, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Ensemble combination with threshold influence
        threshold_weight = min(0.7, threshold * 10)  # Stronger threshold influence for low values
        adaptive_weight = max(0.3, 1.0 - threshold * 8)  # Stronger adaptive for low thresholds
        
        combined = np.maximum(
            np.maximum(
                otsu_mask * 0.2,
                threshold_mask * threshold_weight
            ),
            np.maximum(
                percentile_mask * 0.3,
                adaptive_mask * adaptive_weight
            )
        )
        
        return combined.astype(np.uint8), combined
    
    def _nnunet_segment_with_threshold(self, image, threshold):
        """nnU-Net segmentation with custom threshold"""
        # Use the existing nnU-Net approach but with custom threshold
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        else:
            gray = image.copy()
        
        # Apply preprocessing
        gray = self.enhanced_preprocessing(gray)
        
        # Multi-scale with custom threshold
        binary_mask_1 = self._nnunet_threshold_segment(gray, threshold)
        gray_half = cv2.resize(gray, (gray.shape[1]//2, gray.shape[0]//2))
        binary_mask_half = self._nnunet_threshold_segment(gray_half, threshold * 0.8)
        binary_mask_2 = cv2.resize(binary_mask_half, (gray.shape[1], gray.shape[0]))
        
        # Combine
        combined_mask = np.maximum(binary_mask_1 * 0.7, binary_mask_2 * 0.3)
        binary_mask = (combined_mask > threshold).astype(np.uint8)
        
        return binary_mask, combined_mask
    
    def _lungs_seg_segment_with_threshold(self, image, threshold):
        """lungs-segmentation with custom threshold"""
        try:
            from lungs_segmentation import inference
            processed_image, masks = inference.inference(self.seg_model, image, threshold)
            
            if len(masks) >= 2:
                combined_mask = masks[0] + masks[1]
            elif len(masks) == 1:
                combined_mask = masks[0]
            else:
                return self._enhanced_fallback_segmentation_with_threshold(image, threshold)
            
            binary_mask = (combined_mask > threshold).astype(np.uint8)
            return binary_mask, combined_mask
        except:
            return self._enhanced_fallback_segmentation_with_threshold(image, threshold)
    
    def ensemble_segmentation(self, image):
        """Multi-threshold ensemble approach for maximum sensitivity"""
        print(f"{ANSI['C']}🗳️ Running ensemble segmentation with thresholds: {ENSEMBLE_THRESHOLDS}...{ANSI['W']}")
        
        # Apply enhanced preprocessing once
        preprocessed = self.enhanced_preprocessing(image)
        
        results = []
        prob_results = []
        
        for i, thresh in enumerate(ENSEMBLE_THRESHOLDS):
            print(f"{ANSI['C']}  🎛️ Threshold {i+1}/{len(ENSEMBLE_THRESHOLDS)}: {thresh}...{ANSI['W']}")
            
            binary_mask, prob_mask = self._segment_with_threshold(preprocessed, thresh)
            results.append(binary_mask)
            prob_results.append(prob_mask)
        
        # Voting ensemble: pixel is lung if enough thresholds agree
        ensemble_mask = np.mean(results, axis=0)
        final_mask = (ensemble_mask >= ENSEMBLE_VOTING_THRESHOLD).astype(np.uint8)
        
        # Enhanced post-processing for ensemble result
        final_mask = self._ensemble_postprocess(final_mask)
        
        # Calculate agreement statistics
        agreement_ratio = np.sum(ensemble_mask >= ENSEMBLE_VOTING_THRESHOLD) / ensemble_mask.size
        print(f"{ANSI['G']}✅ Ensemble complete: {agreement_ratio:.3f} agreement ratio, voting threshold: {ENSEMBLE_VOTING_THRESHOLD}{ANSI['W']}")
        
        return final_mask, ensemble_mask
    
    def _ensemble_postprocess(self, binary_mask):
        """Enhanced post-processing for ensemble results"""
        # More aggressive morphological operations for ensemble results
        # Close gaps between lung regions
        kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel_close)
        
        # Remove small noise
        kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel_open)
        
        # Fill holes within lung regions
        kernel_fill = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (30, 30))
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel_fill)
        
        return binary_mask
    
    def segment_lungs(self, image):
        """Enhanced lung segmentation with ensemble and preprocessing"""
        if self.model is None:
            return self._enhanced_fallback_segmentation(image)
        
        try:
            if USE_ENSEMBLE_SEGMENTATION:
                return self.ensemble_segmentation(image)
            else:
                # Apply enhanced preprocessing even for single model
                preprocessed = self.enhanced_preprocessing(image)
                
                if self.model == 'torchxray':
                    return self._torchxray_segment(preprocessed)
                elif self.model == 'lungs_seg':
                    return self._lungs_seg_segment(preprocessed)
                elif self.model == 'nnunet':
                    return self._nnunet_segment(preprocessed)
        except Exception as e:
            print(f"{ANSI['Y']}⚠️ Segmentation failed: {e}, using enhanced fallback{ANSI['W']}")
            return self._enhanced_fallback_segmentation(image)

print(f"{ANSI['G']}✅ Enhanced segmentation methods loaded{ANSI['W']}")


In [ ]:
# This cell now contains only the initialization code

# Initialize the ENHANCED segmentation pipeline
if USE_LUNG_SEGMENTATION:
    if SEGMENTATION_MODEL in available_models:
        lung_segmenter = EnhancedLungSegmentation(SEGMENTATION_MODEL)
    elif 'torchxrayvision' in available_models:
        lung_segmenter = EnhancedLungSegmentation('torchxrayvision')
    elif 'nnunet' in available_models:
        lung_segmenter = EnhancedLungSegmentation('nnunet')
    elif 'lungs_segmentation' in available_models:
        lung_segmenter = EnhancedLungSegmentation('lungs_segmentation')
    else:
        lung_segmenter = EnhancedLungSegmentation('fallback')
    print(f"{ANSI['C']}🎯 ENHANCED lung segmentation initialized with {SEGMENTATION_MODEL}{ANSI['W']}")
    print(f"{ANSI['M']}🆕 Features: Ensemble={USE_ENSEMBLE_SEGMENTATION}, Enhanced Preprocessing={USE_ENHANCED_PREPROCESSING}{ANSI['W']}")
else:
    lung_segmenter = None
    print(f"{ANSI['Y']}⚠️ Lung segmentation disabled{ANSI['W']}")


In [ ]:
def process_image_with_segmentation(image_array, file_id):
    """Process image with professional lung segmentation and improved cropping"""
    if lung_segmenter is None:
        return image_array
    
    try:
        print(f"{ANSI['B']}🫁 ENHANCED segmenting lungs for {file_id}...{ANSI['W']}")
        if USE_ENSEMBLE_SEGMENTATION:
            print(f"{ANSI['M']}🗳️ Using ensemble voting with {len(ENSEMBLE_THRESHOLDS)} thresholds{ANSI['W']}")
        if USE_ENHANCED_PREPROCESSING:
            print(f"{ANSI['C']}🔍 Applying research-based preprocessing{ANSI['W']}")
        
        # Get enhanced lung segmentation
        binary_mask, prob_mask = lung_segmenter.segment_lungs(image_array)
        
        # Validate segmentation quality
        total_pixels = binary_mask.shape[0] * binary_mask.shape[1]
        lung_pixels = np.sum(binary_mask)
        lung_ratio = lung_pixels / total_pixels
        
        print(f"{ANSI['C']}📊 Lung area detected: {lung_ratio:.3f} of image{ANSI['W']}")
        
        # Quality check with improved thresholds
        if lung_ratio < MIN_LUNG_AREA_RATIO:
            print(f"{ANSI['Y']}⚠️ Detected area too small ({lung_ratio:.3f} < {MIN_LUNG_AREA_RATIO}), using original{ANSI['W']}")
            return image_array  # Return original
        
        if lung_ratio > MAX_LUNG_AREA_RATIO:
            print(f"{ANSI['Y']}⚠️ Detected area too large ({lung_ratio:.3f} > {MAX_LUNG_AREA_RATIO}), using original{ANSI['W']}")
            return image_array  # Return original
        
        # Find bounding box of lung regions - GUARANTEED FULL COVERAGE
        coords = np.column_stack(np.where(binary_mask > 0))
        if len(coords) == 0:
            print(f"{ANSI['Y']}⚠️ No lung coordinates found{ANSI['W']}")
            return image_array
        
        y_min, x_min = coords.min(axis=0)
        y_max, x_max = coords.max(axis=0)
        
        # Add generous safety margin to ensure ALL lung pixels are included (increased for containment)
        safety_margin = 20  # DOUBLED from 10px to ensure complete containment
        y_min = max(0, y_min - safety_margin)
        x_min = max(0, x_min - safety_margin)
        y_max = min(image_array.shape[0], y_max + safety_margin)
        x_max = min(image_array.shape[1], x_max + safety_margin)
        
        print(f"{ANSI['C']}📦 Lung bounding box: ({y_min},{x_min}) to ({y_max},{x_max}) with {safety_margin}px safety margin{ANSI['W']}")
        
        # Add generous padding to ensure lungs are fully included
        h, w = image_array.shape[:2]
        padding = LUNG_CROP_PADDING
        
        # Calculate padded boundaries
        y_min_padded = max(0, y_min - padding)
        x_min_padded = max(0, x_min - padding)
        y_max_padded = min(h, y_max + padding)
        x_max_padded = min(w, x_max + padding)
        
        # Ensure minimum crop size to avoid over-cropping
        crop_height = y_max_padded - y_min_padded
        crop_width = x_max_padded - x_min_padded
        min_dimension = min(h, w) * 0.5  # At least 50% of smallest dimension
        
        if crop_height < min_dimension or crop_width < min_dimension:
            print(f"{ANSI['Y']}⚠️ Crop too small ({crop_height}x{crop_width}), using more conservative crop{ANSI['W']}")
            # Use more conservative padding
            center_y, center_x = (y_min + y_max) // 2, (x_min + x_max) // 2
            half_size = int(min_dimension // 2)
            
            y_min_padded = max(0, center_y - half_size)
            x_min_padded = max(0, center_x - half_size)
            y_max_padded = min(h, center_y + half_size)
            x_max_padded = min(w, center_x + half_size)
        
        # 🔍 FINAL VALIDATION: Ensure ALL lung pixels are contained within crop rectangle
        lung_coords = np.column_stack(np.where(binary_mask > 0))
        if len(lung_coords) > 0:
            actual_y_min, actual_x_min = lung_coords.min(axis=0)
            actual_y_max, actual_x_max = lung_coords.max(axis=0)
            
            # Check if any lung pixels extend beyond crop rectangle
            if (actual_y_min < y_min_padded or actual_x_min < x_min_padded or 
                actual_y_max >= y_max_padded or actual_x_max >= x_max_padded):
                
                print(f"{ANSI['Y']}⚠️ Lung pixels extend beyond crop, expanding rectangle...{ANSI['W']}")
                print(f"   Lung bounds: ({actual_y_min},{actual_x_min}) to ({actual_y_max},{actual_x_max})")
                print(f"   Crop bounds: ({y_min_padded},{x_min_padded}) to ({y_max_padded},{x_max_padded})")
                
                # Expand crop rectangle to fully contain all lung pixels + extra safety
                expand_margin = 15  # Extra margin for safety
                y_min_padded = max(0, actual_y_min - expand_margin)
                x_min_padded = max(0, actual_x_min - expand_margin)
                y_max_padded = min(h, actual_y_max + expand_margin)
                x_max_padded = min(w, actual_x_max + expand_margin)
                
                print(f"{ANSI['G']}✅ Expanded crop: ({y_min_padded},{x_min_padded}) to ({y_max_padded},{x_max_padded}){ANSI['W']}")
            else:
                print(f"{ANSI['G']}✅ All lung pixels contained within crop rectangle{ANSI['W']}")
        
        # Crop the image - KEEP ORIGINAL IMAGE CONTENT (No gradient filling)
        if len(image_array.shape) == 3:
            cropped = image_array[y_min_padded:y_max_padded, x_min_padded:x_max_padded, :]
        else:
            cropped = image_array[y_min_padded:y_max_padded, x_min_padded:x_max_padded]
        
        print(f"{ANSI['G']}✅ Lung-guided cropping complete - original image content preserved{ANSI['W']}")
        print(f"{ANSI['C']}   🫁 Segmentation used for crop boundaries only, original image kept intact{ANSI['W']}")
        
        # Calculate area reduction
        original_area = h * w
        cropped_area = (y_max_padded - y_min_padded) * (x_max_padded - x_min_padded)
        area_reduction = cropped_area / original_area
        
        print(f"{ANSI['G']}✅ ENHANCED cropping: {area_reduction:.2f} area reduction{ANSI['W']}")
        print(f"{ANSI['B']}   Original: {h}x{w} → Cropped: {y_max_padded-y_min_padded}x{x_max_padded-x_min_padded}{ANSI['W']}")
        print(f"{ANSI['C']}   INCREASED padding: {padding}px (was 120px, now {LUNG_CROP_PADDING}px for better coverage){ANSI['W']}")
        
        # Save segmentation mask if requested - FIXED MASK SAVING
        if SAVE_SEGMENTATION_MASKS:
            print(f"{ANSI['C']}💾 Saving segmentation masks...{ANSI['W']}")
            
            # Ensure masks directory exists
            os.makedirs(MASKS_PATH, exist_ok=True)
            print(f"{ANSI['B']}   📁 Masks directory: {MASKS_PATH}{ANSI['W']}")
            
            # Save clean binary mask (like reference image - simple contours)
            mask_path = os.path.join(MASKS_PATH, f"{file_id}_mask.png")
            mask_image = (binary_mask * 255).astype(np.uint8)
            mask_saved = cv2.imwrite(mask_path, mask_image)
            
            # Save overlay with correct hierarchy: RESIZE crop contains SEGMENTATION crop
            overlay_path = os.path.join(MASKS_PATH, f"{file_id}_overlay.png")
            overlay = image_array.copy()
            if len(overlay.shape) == 2:
                overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
            
            # Calculate GREEN final crop/resize rectangle FIRST (this should contain everything)
            img_height, img_width = overlay.shape[:2]
            
            # Find lung center for positioning the resize crop
            lung_center_y = (y_min_padded + y_max_padded) // 2
            lung_center_x = (x_min_padded + x_max_padded) // 2
            
            # Calculate resize crop dimensions based on TARGET_SIZE
            target_aspect_ratio = TARGET_SIZE[0] / TARGET_SIZE[1]  # width/height
            
            # Make resize crop large enough to contain the lung area, but respect aspect ratio
            lung_width = x_max_padded - x_min_padded
            lung_height = y_max_padded - y_min_padded
            
            # Calculate minimum size needed to contain lungs, then expand if needed
            min_width = lung_width + 40  # Extra margin
            min_height = lung_height + 40
            
            # Ensure aspect ratio is maintained
            if min_width / min_height > target_aspect_ratio:
                # Width is limiting factor
                resize_width = min_width
                resize_height = int(resize_width / target_aspect_ratio)
            else:
                # Height is limiting factor  
                resize_height = min_height
                resize_width = int(resize_height * target_aspect_ratio)
            
            # Center resize crop on lung center, but keep within image bounds
            resize_x_min = max(0, lung_center_x - resize_width // 2)
            resize_y_min = max(0, lung_center_y - resize_height // 2)
            resize_x_max = min(img_width, resize_x_min + resize_width)
            resize_y_max = min(img_height, resize_y_min + resize_height)
            
            # Adjust if we hit image boundaries
            if resize_x_max == img_width:
                resize_x_min = img_width - resize_width
            if resize_y_max == img_height:
                resize_y_min = img_height - resize_height
                
            # Ensure non-negative coordinates
            resize_x_min = max(0, resize_x_min)
            resize_y_min = max(0, resize_y_min)
            
            # Create a mask for areas OUTSIDE the resize crop (these will be darkened)
            outside_resize_mask = np.ones((img_height, img_width), dtype=bool)
            outside_resize_mask[resize_y_min:resize_y_max, resize_x_min:resize_x_max] = False
            
            # Apply 50% darkening ONLY to areas outside the resize crop
            overlay[outside_resize_mask] = (overlay[outside_resize_mask] * 0.5).astype(np.uint8)
            
            # Create RED lung visualization with separate border and fill opacities
            lung_areas = binary_mask > 0
            
            if np.any(lung_areas):
                # Step 1: Apply DARK RED FILL with 50% opacity
                lung_fill_colored = np.zeros_like(overlay)
                lung_fill_colored[lung_areas] = [0, 0, 180]  # Dark Red in BGR
                
                overlay[lung_areas] = cv2.addWeighted(
                    overlay[lung_areas], 1.0 - LUNG_FILL_OPACITY, 
                    lung_fill_colored[lung_areas], LUNG_FILL_OPACITY, 0
                )
                
                # Step 2: Find lung borders and apply stronger dark red with 75% opacity
                # Create border mask by finding edges
                lung_mask_uint8 = (binary_mask * 255).astype(np.uint8)
                
                # Find contours for border
                contours, _ = cv2.findContours(lung_mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                # Create border mask
                border_mask = np.zeros_like(binary_mask, dtype=np.uint8)
                cv2.drawContours(border_mask, contours, -1, 1, thickness=4)  # 4px thick border
                
                # Apply DARK RED BORDER with 75% opacity
                border_areas = border_mask > 0
                if np.any(border_areas):
                    lung_border_colored = np.zeros_like(overlay)
                    lung_border_colored[border_areas] = [0, 0, 180]  # Dark Red in BGR
                    
                    overlay[border_areas] = cv2.addWeighted(
                        overlay[border_areas], 1.0 - LUNG_BORDER_OPACITY, 
                        lung_border_colored[border_areas], LUNG_BORDER_OPACITY, 0
                    )
            
            # Draw ORANGE corner brackets for segmentation boundaries (instead of full rectangle)
            def draw_corner_brackets(img, x1, y1, x2, y2, color, thickness=3, length=30):
                """Draw corner brackets at the corners of a rectangle"""
                # Top-left corner
                cv2.line(img, (x1, y1), (x1 + length, y1), color, thickness)  # Horizontal
                cv2.line(img, (x1, y1), (x1, y1 + length), color, thickness)  # Vertical
                
                # Top-right corner
                cv2.line(img, (x2 - length, y1), (x2, y1), color, thickness)  # Horizontal
                cv2.line(img, (x2, y1), (x2, y1 + length), color, thickness)  # Vertical
                
                # Bottom-left corner
                cv2.line(img, (x1, y2 - length), (x1, y2), color, thickness)  # Vertical
                cv2.line(img, (x1, y2), (x1 + length, y2), color, thickness)  # Horizontal
                
                # Bottom-right corner
                cv2.line(img, (x2, y2 - length), (x2, y2), color, thickness)  # Vertical
                cv2.line(img, (x2 - length, y2), (x2, y2), color, thickness)  # Horizontal
            
            # Find actual segmentation boundaries (without padding) for orange corners
            lung_coords = np.where(binary_mask > 0)
            if len(lung_coords[0]) > 0:
                actual_y_min = np.min(lung_coords[0])  # Top edge of lungs
                actual_y_max = np.max(lung_coords[0])  # Bottom edge of lungs  
                actual_x_min = np.min(lung_coords[1])  # Left edge of lungs
                actual_x_max = np.max(lung_coords[1])  # Right edge of lungs
                
                # Draw GREEN corner brackets at EXACT segmentation boundaries (no padding)
                draw_corner_brackets(overlay, actual_x_min, actual_y_min, actual_x_max, actual_y_max, (0, 255, 0), 3, 40)
            else:
                print(f"{ANSI['Y']}⚠️ No lung areas found for corner brackets{ANSI['W']}")
            
            # Draw CYAN resize crop rectangle (contains everything) - 1px contour line
            cv2.rectangle(overlay, (resize_x_min, resize_y_min), (resize_x_max, resize_y_max), (255, 255, 0), 1)
            
            # Create legend with BLACK BACKGROUND and larger text
            img_height, img_width = overlay.shape[:2]
            legend_height = 120
            legend_width = min(700, img_width - 20)  # Ensure legend fits within image bounds
            legend_y_start = img_height - legend_height - 10
            
            # Create black background with 75% opacity
            legend_background = np.zeros((legend_height, legend_width, 3), dtype=np.uint8)
            legend_area = overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width]
            
            # Blend background with 75% opacity (25% transparency)
            overlay[legend_y_start:legend_y_start + legend_height, 10:10 + legend_width] = cv2.addWeighted(
                legend_area, 0.25, legend_background, 0.75, 0
            )
            
            # Add legend text with larger font
            text_y = legend_y_start + 25
            font_scale = 0.7  # Larger font
            font_thickness = 2
            
            cv2.putText(overlay, f"DARK RED = Lung segmentation (Fill: {int(LUNG_FILL_OPACITY*100)}%, Border: {int(LUNG_BORDER_OPACITY*100)}%)", 
                       (20, text_y), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 0, 180), font_thickness)
            
            cv2.putText(overlay, "GREEN = Segmentation corner brackets", 
                       (20, text_y + 30), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)
            
            cv2.putText(overlay, f"CYAN = Final resize crop {TARGET_SIZE[0]}x{TARGET_SIZE[1]}", 
                       (20, text_y + 60), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 0), font_thickness)
            
            # Add feature information at bottom with smaller font
            feature_text = f"Threshold: {LUNG_SEGMENTATION_THRESHOLD}"
            if USE_ENSEMBLE_SEGMENTATION:
                feature_text += f" | Ensemble: {len(ENSEMBLE_THRESHOLDS)} votes"
            if USE_ENHANCED_PREPROCESSING:
                feature_text += " | Enhanced preprocessing"
            cv2.putText(overlay, feature_text, (20, text_y + 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            overlay_saved = cv2.imwrite(overlay_path, overlay)
            
            # Verify files were saved
            if mask_saved and overlay_saved:
                print(f"{ANSI['G']}   ✅ Saved {file_id}_mask.png ({os.path.getsize(mask_path)} bytes){ANSI['W']}")
                print(f"{ANSI['G']}   ✅ Saved {file_id}_overlay.png ({os.path.getsize(overlay_path)} bytes){ANSI['W']}")
            else:
                print(f"{ANSI['R']}   ❌ Failed to save masks for {file_id}{ANSI['W']}")
        else:
            print(f"{ANSI['Y']}⚠️ Mask saving disabled (SAVE_SEGMENTATION_MASKS = {SAVE_SEGMENTATION_MASKS}){ANSI['W']}")
        
        return cropped
        
    except Exception as e:
        print(f"{ANSI['Y']}⚠️ Segmentation failed for {file_id}: {e}{ANSI['W']}")
        return image_array

print(f"{ANSI['G']}✅ Enhanced image processing functions loaded{ANSI['W']}")


In [ ]:
# 🔍 VERIFICATION: Classes Definition Status

print("="*60)
print(f"{ANSI['G']}✅ CLASS DEFINITIONS STATUS:{ANSI['W']}")
print("="*60)

try:
    # Test base class
    test_base = PreTrainedLungSegmentation.__name__
    print(f"✅ PreTrainedLungSegmentation: DEFINED")
    
    # Test enhanced class  
    test_enhanced = EnhancedLungSegmentation.__name__
    print(f"✅ EnhancedLungSegmentation: DEFINED")
    
    # Test inheritance
    if issubclass(EnhancedLungSegmentation, PreTrainedLungSegmentation):
        print(f"✅ Inheritance: EnhancedLungSegmentation → PreTrainedLungSegmentation")
    
    print(f"\n{ANSI['G']}🎉 ALL CLASS DEFINITIONS SUCCESSFUL!{ANSI['W']}")
    print(f"{ANSI['C']}Ready to proceed with enhanced lung segmentation.{ANSI['W']}")
    
except NameError as e:
    print(f"❌ Class definition error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

print("="*60)


In [ ]:
# 🎉 ENHANCED SEGMENTATION FEATURES SUMMARY

print("="*80)
print(f"{ANSI['M']}🌟 ENHANCED LUNG SEGMENTATION V1.3 FEATURES IMPLEMENTED:{ANSI['W']}")
print("="*80)

print(f"\n{ANSI['G']}📈 SENSITIVITY IMPROVEMENTS:{ANSI['W']}")
print(f"   • Threshold: {LUNG_SEGMENTATION_THRESHOLD} (was 0.1, now ULTRA sensitive)")
print(f"   • Min area ratio: {MIN_LUNG_AREA_RATIO} (was 0.02, now ultra-permissive)")
print(f"   • Padding: {LUNG_CROP_PADDING}px (was 120px, now generous coverage)")

print(f"\n{ANSI['M']}🗳️ ENSEMBLE VOTING:{ANSI['W']}")
print(f"   • Enabled: {USE_ENSEMBLE_SEGMENTATION}")
print(f"   • Thresholds: {ENSEMBLE_THRESHOLDS}")
print(f"   • Voting threshold: {ENSEMBLE_VOTING_THRESHOLD} (40% agreement)")

print(f"\n{ANSI['C']}🔍 ENHANCED PREPROCESSING:{ANSI['W']}")
print(f"   • Enabled: {USE_ENHANCED_PREPROCESSING}")
print(f"   • CLAHE: Clip limit {CLAHE_CLIP_LIMIT}, Tile size {CLAHE_TILE_SIZE}")
print(f"   • Bilateral filter: {USE_BILATERAL_FILTER} (edge-preserving)")
print(f"   • Histogram equalization: Enabled")

print(f"\n{ANSI['B']}🧠 RESEARCH-BASED FEATURES:{ANSI['W']}")
print("   • Multi-threshold ensemble (Nature Scientific Reports)")
print("   • Enhanced CLAHE preprocessing (optimized for lung imaging)")
print("   • Bilateral filtering for noise reduction")
print("   • Adaptive thresholding with percentile mapping")
print("   • Advanced morphological post-processing")

print(f"\n{ANSI['G']}🖼️ IMAGE PROCESSING:{ANSI['W']}")
print("   • Lung-guided cropping (segmentation defines boundaries)")
print("   • Original image content preserved (no gradient filling)")
print("   • Enhanced segmentation masks saved to Masks folder")
print(f"   • Mask saving: {SAVE_SEGMENTATION_MASKS}")

print(f"\n{ANSI['R']}🎨 OVERLAY VISUALIZATION:{ANSI['W']}")
print(f"   • Dark red lung segmentation with dual opacity:")
print(f"     - Fill: {int(LUNG_FILL_OPACITY*100)}% opacity (softer background)")
print(f"     - Border: {int(LUNG_BORDER_OPACITY*100)}% opacity (defined edges)")
print("   • Green corner brackets (exact segmentation boundaries)")
print("   • Cyan resize crop rectangle (final boundaries)")
print("   • Black legend background with 75% opacity")
print(f"   • Configurable opacities: LUNG_FILL_OPACITY = {LUNG_FILL_OPACITY}, LUNG_BORDER_OPACITY = {LUNG_BORDER_OPACITY}")

print(f"\n{ANSI['Y']}🎯 MODEL CAPABILITIES:{ANSI['W']}")
if 'torchxrayvision' in available_models:
    print("   ✅ TorchXRayVision: Pre-trained on NIH, CheXpert, MIMIC datasets")
if 'nnunet' in available_models:
    print("   ✅ nnU-Net: State-of-the-art medical segmentation framework")
if 'lungs_segmentation' in available_models:
    print("   ✅ lungs-segmentation: Specialized lung segmentation models")
print("   ✅ Enhanced fallback: Multi-method ensemble approach")

print(f"\n{ANSI['G']}📊 EXPECTED IMPROVEMENTS:{ANSI['W']}")
print("   • Detection sensitivity: Significantly increased")
print("   • Lung coverage: More complete lung region capture")
print("   • Edge detection: Better preservation of lung boundaries")
print("   • Noise robustness: Improved handling of image artifacts")
print("   • Quality consistency: More reliable across different image types")

print(f"\n{ANSI['C']}🔧 CONFIGURATION:{ANSI['W']}")
print("   • All parameters configurable at top of notebook")
print("   • Real-time threshold adjustment support")
print("   • Individual feature enable/disable options")

print("="*80)
print(f"{ANSI['M']}🚀 READY FOR MAXIMUM SENSITIVITY LUNG SEGMENTATION!{ANSI['W']}")
print("="*80)


In [ ]:
# 🚀 Main Processing Pipeline

# Load CSV data
try:
    user_info = a3conn.getUserInfos()
    print(f"{ANSI['G']}🔐 ArchiMed User Info{ANSI['W']}")
    print(f"User info: {user_info}")
    
    # Load CSV
    csv_path = os.path.join(CSV_FOLDER, CSV_LABELS_FILE)
    df = pd.read_csv(csv_path, sep=CSV_SEPARATOR)
    print(f"{ANSI['G']}✅ Loaded CSV with {len(df)} rows{ANSI['W']}")
    
    # Check for FileID column (handle different naming conventions)
    file_id_column = None
    for col in ['FileID', 'file_id', 'File_ID']:
        if col in df.columns:
            file_id_column = col
            break
    
    if file_id_column is None:
        print(f"{ANSI['R']}❌ No FileID column found in CSV{ANSI['W']}")
        raise ValueError("FileID column not found")
    
    print(f"{ANSI['C']}📊 Available columns: {list(df.columns)}{ANSI['W']}")
    
    # Get file IDs to download
    file_ids = df[file_id_column].dropna().unique()
    total_files = len(file_ids)
    
    print(f"{ANSI['M']}🚀 Starting enhanced download with pre-trained lung segmentation{ANSI['W']}")
    print(f"Total files to process: {total_files}")
    print(f"Destination: {DOWNLOAD_PATH}")
    print(f"🫁 Lung segmentation: {'ENABLED' if USE_LUNG_SEGMENTATION else 'DISABLED'}")
    
    # Download files
    downloaded_files = []
    
    for i, file_id in enumerate(file_ids):
        progress = ((i + 1) / total_files) * 100
        # Convert numpy.int64 to string for API compatibility
        file_id_str = str(file_id)
        print(f"{ANSI['B']}⬇️ Downloading file {file_id_str} (Progress: {progress:.1f}% - {i+1}/{total_files}) from ArchiMed{ANSI['W']}")
        
        try:
            # Define output path - FLAT STRUCTURE (no subfolders)
            dicom_file_path = os.path.join(DOWNLOAD_PATH, f"{file_id}.dcm")
            # Create download directory if it doesn't exist
            os.makedirs(DOWNLOAD_PATH, exist_ok=True)
            
            # Check if the file already exists
            if os.path.exists(dicom_file_path):
                print(f"{ANSI['Y']}File {file_id} already exists, skipping download{ANSI['W']}")
                downloaded_files.append(dicom_file_path)
                continue
            
            # Download using the WORKING v1.1 pattern - MODIFIED for flat structure
            result = a3conn.downloadFile(
                int(file_id_str),  # Convert back to int as API expects
                asStream=False,
                destDir=DOWNLOAD_PATH,  # Use main directory directly
                filename=f"{file_id_str}.dcm",
                inWorklist=False
            )
            
            if result and os.path.exists(dicom_file_path):
                downloaded_files.append(dicom_file_path)
                print(f"{ANSI['G']}✅ Successfully downloaded: {dicom_file_path}{ANSI['W']}")
            else:
                print(f"{ANSI['Y']}⚠️ Download result unclear for {file_id_str}{ANSI['W']}")
        except Exception as e:
            print(f"{ANSI['Y']}⚠️ Failed to download {file_id_str}: {e}{ANSI['W']}")
    
    print(f"{ANSI['G']}✅ Downloaded {len(downloaded_files)} files successfully{ANSI['W']}")
    
except Exception as e:
    print(f"{ANSI['R']}❌ Setup failed: {e}{ANSI['W']}")
    downloaded_files = []


In [ ]:
# 🏥 Enhanced DICOM Conversion with Pre-trained Lung Segmentation

def convert_dicom_to_image_with_segmentation(dicom_path, output_path, target_size=TARGET_SIZE):
    """Enhanced DICOM conversion with professional lung segmentation"""
    try:
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        
        # Read DICOM file
        dicom_data = pydicom.dcmread(dicom_path)
        
        # Extract image data
        image_array = dicom_data.pixel_array
        print(f"{ANSI['C']}📁 Processing {file_id}: {image_array.shape}{ANSI['W']}")
        
        # Handle different photometric interpretations
        if hasattr(dicom_data, 'PhotometricInterpretation'):
            if dicom_data.PhotometricInterpretation == 'MONOCHROME1':
                image_array = np.max(image_array) - image_array
        
        # Normalize to 0-255 range
        if image_array.max() > 255:
            image_array = ((image_array - image_array.min()) / 
                          (image_array.max() - image_array.min()) * 255).astype(np.uint8)
        else:
            image_array = image_array.astype(np.uint8)
        
        # Apply lung segmentation and cropping
        processed_image = process_image_with_segmentation(image_array, file_id)
        
        # Convert to PIL Image
        if len(processed_image.shape) == 2:
            pil_image = Image.fromarray(processed_image, mode='L')
        else:
            pil_image = Image.fromarray(processed_image)
        
        # Enhanced aspect ratio preservation - NO BLACK MARGINS
        if PRESERVE_ASPECT_RATIO:
            print(f"{ANSI['C']}🖼️ Preserving aspect ratio by intelligent cropping (no black margins)...{ANSI['W']}")
            
            # Get current and target dimensions
            current_width, current_height = pil_image.size
            target_width, target_height = target_size
            
            # Calculate aspect ratios
            current_ratio = current_width / current_height
            target_ratio = target_width / target_height
            
            print(f"{ANSI['C']}   Current: {current_width}x{current_height} (ratio: {current_ratio:.3f}){ANSI['W']}")
            print(f"{ANSI['C']}   Target: {target_width}x{target_height} (ratio: {target_ratio:.3f}){ANSI['W']}")
            
            # Crop to match target aspect ratio (center crop to avoid black margins)
            if current_ratio > target_ratio:
                # Current image is wider - crop width to match target ratio
                new_width = int(current_height * target_ratio)
                new_height = current_height
                left = (current_width - new_width) // 2
                top = 0
                right = left + new_width
                bottom = current_height
                print(f"{ANSI['B']}   📏 Cropping width: {current_width} → {new_width} (centered crop){ANSI['W']}")
            else:
                # Current image is taller - crop height to match target ratio  
                new_width = current_width
                new_height = int(current_width / target_ratio)
                left = 0
                top = (current_height - new_height) // 2
                right = current_width
                bottom = top + new_height
                print(f"{ANSI['B']}   📏 Cropping height: {current_height} → {new_height} (centered crop){ANSI['W']}")
            
            # Apply the crop
            pil_image = pil_image.crop((left, top, right, bottom))
            print(f"{ANSI['G']}   ✅ Cropped to: {pil_image.size[0]}x{pil_image.size[1]} (ratio: {pil_image.size[0]/pil_image.size[1]:.3f}){ANSI['W']}")
            
            # Now resize to exact target size (no black margins needed)
            pil_image = pil_image.resize(target_size, Image.Resampling.LANCZOS)
            print(f"{ANSI['G']}   ✅ Final resize to: {target_width}x{target_height} - NO BLACK MARGINS{ANSI['W']}")
        else:
            print(f"{ANSI['Y']}📐 Stretching to target size (aspect ratio not preserved){ANSI['W']}")
            pil_image = pil_image.resize(target_size, Image.Resampling.LANCZOS)
        
        # Save image
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        pil_image.save(output_path)
        
        return True
        
    except Exception as e:
        print(f"{ANSI['R']}❌ Failed to convert {dicom_path}: {e}{ANSI['W']}")
        return False

# Convert downloaded DICOM files
if CONVERT and downloaded_files:
    print(f"{ANSI['M']}🔄 Converting {len(downloaded_files)} DICOM files with lung segmentation{ANSI['W']}")
    
    # Ensure output directory exists
    os.makedirs(IMAGES_PATH, exist_ok=True)
    
    converted_count = 0
    
    for dicom_path in tqdm(downloaded_files, desc="Converting DICOMs"):
        file_id = os.path.splitext(os.path.basename(dicom_path))[0]
        output_path = os.path.join(IMAGES_PATH, f"{file_id}.png")
        
        if convert_dicom_to_image_with_segmentation(dicom_path, output_path):
            converted_count += 1
    
    print(f"{ANSI['G']}✅ Successfully converted {converted_count}/{len(downloaded_files)} files{ANSI['W']}")
    print(f"{ANSI['C']}📂 Images saved to: {IMAGES_PATH}{ANSI['W']}")
    
    if SAVE_SEGMENTATION_MASKS:
        print(f"{ANSI['C']}🎯 Segmentation masks saved to: {MASKS_PATH}{ANSI['W']}")
        
    print(f"{ANSI['M']}🎉 V1.3 Pre-trained lung segmentation processing complete!{ANSI['W']}")
else:
    print(f"{ANSI['Y']}⚠️ No files to convert or conversion disabled{ANSI['W']}")


In [ ]:
# 🖼️ ASPECT RATIO PRESERVATION - NO BLACK MARGINS

print("="*70)
print(f"{ANSI['M']}🖼️ ENHANCED ASPECT RATIO HANDLING{ANSI['W']}")
print("="*70)

print(f"\n{ANSI['G']}✅ PROBLEM SOLVED: No More Black Margins!{ANSI['W']}")
print(f"\n{ANSI['B']}🔧 OLD METHOD (caused black margins):{ANSI['W']}")
print("   1. Resize image to fit within target size")
print("   2. Create new black image of target size") 
print("   3. Paste resized image in center")
print("   4. ❌ Result: Black margins around image")

print(f"\n{ANSI['G']}🆕 NEW METHOD (intelligent cropping):{ANSI['W']}")
print("   1. Calculate current vs target aspect ratios")
print("   2. Center-crop image to match target aspect ratio")
print("   3. Resize cropped image to exact target size")
print("   4. ✅ Result: Perfect fit with NO black margins")

print(f"\n{ANSI['C']}📐 TECHNICAL DETAILS:{ANSI['W']}")
print(f"   • Target size: {TARGET_SIZE}")
print(f"   • Target aspect ratio: {TARGET_SIZE[0]/TARGET_SIZE[1]:.3f}")
print(f"   • Preserve aspect ratio: {PRESERVE_ASPECT_RATIO}")
print("   • Cropping strategy: Center crop (preserves most important content)")
print("   • Resize method: LANCZOS (high-quality)")

print(f"\n{ANSI['Y']}⚡ BENEFITS:{ANSI['W']}")
print("   • No black margins or padding")
print("   • Perfect aspect ratio preservation")  
print("   • All pixels contain actual image data")
print("   • Better utilization of target resolution")
print("   • Cleaner final images for training/inference")

print(f"\n{ANSI['M']}🎯 WHEN CROPPING HAPPENS:{ANSI['W']}")
print("   • If source is wider than target: Crop width (left/right)")
print("   • If source is taller than target: Crop height (top/bottom)")
print("   • Always center-crop to preserve most important content")

print(f"\n{ANSI['R']}🎨 BLACK MARGIN ELIMINATION:{ANSI['W']}")
print("   • Detects background areas around lung tissue")
print("   • Creates intelligent gradient filling based on lung intensity")
print("   • Uses distance transform for natural transitions")
print("   • Adds subtle noise for realistic appearance")
print("   • Never uses pure black (minimum intensity > 30)")

print("="*70)
print(f"{ANSI['G']}🎉 READY TO CONVERT WITH NO BLACK MARGINS GUARANTEED!{ANSI['W']}")
print("="*70)
